## Creating a New AutoCNet Server Project

In [3]:
import os
os.environ['autocnet_config'] = '/home/jlaura/autocnet_projects/test_db.yml'

import autocnet_server
from autocnet_server.db.model import Base

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database

/home/jlaura/autocnet_server/autocnet_server/__init__.py


{'algorithms': {'ring_match': [{'target_points': 25, 'tolerance': 0.01},
   {'target_points': 20, 'tolerance': 0.01},
   {'target_points': 20, 'tolerance': 0.02},
   {'target_points': 15, 'tolerance': 0.025}]},
 'cluster': {'cluster_log_dir': '/home/jlaura/logs',
  'cluster_submission': 'slurm',
  'extractor_memory': 8192,
  'maxfailures': 3,
  'processing_memory': 4000,
  'queue': 'shortall',
  'tmp_scratch_dir': '/scratch/jlaura'},
 'database': {'database_host': 'smalls',
  'database_name': 'jay_test_db',
  'database_password': 'jlaura',
  'database_port': 8001,
  'database_username': 'jlaura',
  'pgbouncer_port': 8003,
  'timeout': 500},
 'database_creation': {'admin': 'jay', 'password': 'abcde'},
 'developer': {'autocnet_path': '/home/jlaura/autocnet',
  'autocnet_server_path': '/home/jlaura/autocnet_server'},
 'directories': {'vrt_dir': '/scratch/jlaura/ctx/vrt'},
 'pyro': {'candidate_graph_host': '0.0.0.0',
  'candidate_graph_port': 8004,
  'candidate_graph_uri': 'ncg',
  'image_

This is the code that creates the database and associated tables for an AutoCNet project. No database names or login information is specified in this file.  This is because everything is being stored in `config.py`.  If we you want to override the values stored in `config.py` this can be done either by modifying that file directly or by modifying the file before executing the below.

In [5]:
config = autocnet_server.config
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(config['database_creation']['admin'],
                                                            config['database_creation']['password'],
                                                            config['database']['database_host'],
                                                            config['database']['database_port'],
                                                            config['database']['database_name']))
if not database_exists(engine.url):
    create_database(engine.url)
# Enable postgis
connection = engine.connect()


result = connection.execute('CREATE EXTENSION postgis')
result = connection.execute("""INSERT into spatial_ref_sys (srid, auth_name, auth_srid, proj4text, srtext) values ( 949900, 'iau2000', 49900, '+proj=longlat +a=3396190 +b=3376200 +no_defs ', 'GEOGCS["Mars 2000",DATUM["D_Mars_2000_IAU_IAG",3396190.0,169.89444722361179]],PRIMEM["Greenwich",0],UNIT["Decimal_Degree",0.0174532925199433]]');""")
result = connection.execute("""ALTER DATABASE {} OWNER TO {}""".format(config['database']['database_name'],config['database']['database_username']))

# Create the tables
Base.metadata.bind = engine
Base.metadata.create_all(engine)